# Functions

CoHDL treats functions similar to macros in C-like programming languages. They are not turned into VHDL functions and instead inlined at their call site. That makes it possible to use arbitrary Python objects as function arguments. The same function can be called with different argument types to produce different VHDL representations.

Function arguments can also have default values. Both positional and keyword arguments are supported.

In [ ]:
from cohdl import Entity, Port, Signed
from cohdl import std

def add(a, b=1):
    return a + b

class ExampleFn(Entity):
    inp_a = Port.input(Signed[16])
    inp_b = Port.input(Signed[16])

    sum_1 = Port.output(Signed[16])
    sum_2 = Port.output(Signed[16])
    sum_3 = Port.output(Signed[16])

    def architecture(self):
        @std.concurrent
        def logic():
            # functions are inlined at their call site
            self.sum_1 <<= add(self.inp_a, self.inp_b)
            # both positional and keyword arguments are supported
            self.sum_2 <<= add(b=self.inp_b, a=self.inp_a)
            # so are defaulted arguments
            self.sum_3 <<= add(self.inp_a)

print(std.VhdlCompiler.to_string(ExampleFn))


Functions can also be recursive as long as the maximal recursion depth is known at compile time. Functions with a variable number of arguments using the `*arg` or `**kwarg` syntax are supported.

In [ ]:
from cohdl import Entity, Port, Signed
from cohdl import std

# functions can have a variable number of arguments
def add_all(first, *rest):
    # compile time recursion is supported
    if len(rest) == 0:
        return first
    else:
        return first + add_all(*rest)

class ExampleFn(Entity):
    inp_a = Port.input(Signed[16])
    inp_b = Port.input(Signed[16])
    inp_c = Port.input(Signed[16])
    inp_d = Port.input(Signed[16])

    sum_1 = Port.output(Signed[16])
    sum_2 = Port.output(Signed[16])

    def architecture(self):
        all_inputs = [self.inp_a, self.inp_b, self.inp_c, self.inp_d]

        @std.concurrent
        def logic():
            self.sum_1 <<= add_all(self.inp_a, self.inp_b, self.inp_c, self.inp_d)

            # unpacking lists is supported
            # this is equivalent to the previous call
            self.sum_2 <<= add_all(*all_inputs)

print(std.VhdlCompiler.to_string(ExampleFn))

## Callback functions

CoHDL functions can take arguments of arbitrary types including other functions.


In [ ]:
from cohdl import Entity, Port, Signed
from cohdl import std

def and_op(a, b):
    return a & b

def or_op(a, b):
    return a | b

# generalization of add_all from the previous example
def binary_fold(binop, first, *rest):
    if len(rest) == 0:
        return first
    else:
        return binop(first, binary_fold(binop, *rest))

class ExampleFn(Entity):
    inp_a = Port.input(Signed[16])
    inp_b = Port.input(Signed[16])
    inp_c = Port.input(Signed[16])
    inp_d = Port.input(Signed[16])

    out_and = Port.output(Signed[16])
    out_or = Port.output(Signed[16])

    def architecture(self):
        all_inputs = [self.inp_a, self.inp_b, self.inp_c, self.inp_d]

        @std.concurrent
        def logic():
            self.out_and <<= binary_fold(and_op, *all_inputs)
            self.out_or <<= binary_fold(or_op, *all_inputs)

print(std.VhdlCompiler.to_string(ExampleFn))